<a href="https://colab.research.google.com/github/lookmeebbear/OpenRS_LDDTNI/blob/main/PythonGEE_LandcoverClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic Land Cover Classification by Google Earth Engine**


Thepchai Srinoi

Department of Survey Engineering

Faculty of Engineering Chulalongkorn University 

-------------------------------------------------------------

การแปลสิ่งปกคลุมดิน (landcover) อัตโนมัติด้วย Random Forest และ Support Vector Machine พื้นที่ฝั่งตะวันตกของแม่น้ำเจ้าพระยา จังหวัดนนทบุรี (บางบัวทอง ปากเกร็ด) และ ปทุมธานี (บางคูวัด) จำแนกระดับเบื้องต้นสุด เป็นสิ่งก่อสร้างมนุษย์สร้างขึ้น พื้นที่เกษตรกรรม และ พื้นที่น้ำ

(1) Install geemap for convert from pandas to gee

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=8e9af02f7f832606672a80408c305b5b8f813490de4cab18f689ace

In [7]:
import geemap
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

(2) Retrieve the satellite image from google earth engine

In [3]:
import ee
# Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lv0SLfyH9M2mylDAf8NvuaQynachW1L-vWDxXTXEC7Y&tc=9rTsAv3lJ4kkFo2zCAS5Epof98Tyj549Arvmyio_Zg8&cc=lgobLa5AFIM9tCf3anQGMhW4sSoe7amlPppnJ-LLNOU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk55FH82JMFVeqqyDRZkbH-i2Cl4LX6QoVIBWKsQyRvP2vvtH29t5VE

Successfully saved authorization token.


In [4]:
import folium
# Install Function for Cloud Mask 
def maskS2clouds(image) :
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0)) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map Visualization : Google Earth Engine
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

In [24]:
# Retrieve satellite image
lat_min, lat_max, lon_min, lon_max = (13.920, 14.000, 100.4, 100.55)
AOI = ee.Geometry.Polygon(
        [[[lon_min, lat_max],
           [lon_min, lat_min],
           [lon_max, lat_min],
           [lon_max, lat_max]]])

fromdate = '2022-01-01'
todate = '2022-02-01'

myimage = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(fromdate,todate).\
            filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',1).\
            filterBounds(AOI).map(maskS2clouds).mosaic()

#True Composite Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=10)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['B6', 'B5', 'B4']},'myimage_falsecolor')
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['B4', 'B3', 'B2']},'myimage_truecolor')
display(map_matched.add_child(folium.LayerControl()))

(3) Download the sample dataset (train, test dataset)

0 - building and artificial place

1 - agriculture

2 - water

In [6]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/lookmeebbear/OpenRS_LDDTNI/main/landcover_samplepoint.csv')
df

,X,Y,class
0,100.533771,13.924564,0
1,100.516601,13.935989,0
2,100.511406,13.930158,0
3,100.512136,13.922949,0
4,100.542498,13.934334,0
...,...,...,...
175,100.439966,13.932960,2
176,100.435115,13.940927,2
177,100.415455,13.947197,2
178,100.429210,13.947346,2


In [43]:
# Map Visualization : Sampling Point 
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=13)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['B6', 'B5', 'B4']},'myimage_falsecolor')
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['B4', 'B3', 'B2']},'myimage_truecolor')

classid = list( set(df['class'].to_list()) )
colorid = ['red','green','blue']
k = -1
for myid in classid :
  k += 1
  df_select = df.loc[ df['class'] == classid[myid] ].copy()
  latitudes = list(df_select.Y)
  longitudes = list(df_select.X)
  for lat, lng in zip(latitudes, longitudes):
    folium.CircleMarker(location = [lat, lng], radius=3, color= colorid[k]).add_to(map_matched)

display(map_matched.add_child(folium.LayerControl()))

(4) Let's do supervised classification ...

In [8]:
from sklearn.model_selection import train_test_split
X = df[df.columns[:2]]
Y = df[df.columns[2]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.50, random_state = 25)

df_train = X_train.join(Y_train)
df_test = X_test.join(Y_test)

In [11]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
#Convert from pandas to featurecollection
train_points = geemap.pandas_to_ee(df_train, latitude='Y', longitude='X')
test_points = geemap.pandas_to_ee(df_test, latitude='Y', longitude='X')

In [25]:
###### CLASSIFICATION #####

#Choose bands
bands = ['B2', 'B3', 'B4', 'B8','B12']

# Overlay the points on the imagery to get training.
training = myimage.sampleRegions(
  collection=train_points,
  properties=['class'],
  scale=30)

# Train a RF,SVM classifier with default parameters.
trained_RF = ee.Classifier.smileRandomForest(10).train(training, 'class', bands)
trained_SVM = ee.Classifier.libsvm().train(training, 'class', bands)

# Classify the image with the same bands used for training.
classified_RF = (myimage.select(bands)).classify(trained_RF)
classified_SVM = (myimage.select(bands)).classify(trained_SVM)

# Accuracy Ascessment
testing = myimage.sampleRegions(collection=test_points, properties=['class'], scale=30) #get raster value first !!!!
testResults_RF = (testing.classify(trained_RF)) #find answer from train actor
testAccuracy_RF = (testResults_RF).errorMatrix('class', 'classification') 

testResults_SVM = (testing.classify(trained_SVM)) #find answer from train actor
testAccuracy_SVM = (testResults_SVM).errorMatrix('class', 'classification') 


print('Random Forest')
print('Validation error matrix: ', testAccuracy_RF.getInfo())
print('Validation overall accuracy: ', testAccuracy_RF.accuracy().getInfo())

print('Support Vector Machine')
print('Validation error matrix: ', testAccuracy_SVM.getInfo())
print('Validation overall accuracy: ', testAccuracy_SVM.accuracy().getInfo())

Random Forest
Validation error matrix:  [[27, 3, 0], [4, 25, 1], [1, 1, 27]]
Validation overall accuracy:  0.8876404494382022
Support Vector Machine
Validation error matrix:  [[19, 9, 2], [1, 26, 3], [0, 0, 29]]
Validation overall accuracy:  0.8314606741573034


In [26]:
#Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=12)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['B4', 'B3', 'B2']},'myimage_truecolor')
map_matched.add_ee_layer(classified_SVM,{'min':0, 'max':2, 'palette':['red', 'green', 'blue']},'myimage_classy_SVM')
map_matched.add_ee_layer(classified_RF,{'min':0, 'max':2, 'palette':['red', 'green', 'blue']},'myimage_classy_RF')
display(map_matched.add_child(folium.LayerControl()))

# **THE END**

31 Mar 2022 .... Thepchai Srinoi

Update for LDD Training 29 Apr 2023 ... Thepchai Srinoi